# Necessary Tools and Services

In [2]:
! pip install pandas
! pip install matplotlib
! pip install numpy
! pip install seaborn
! pip install unzip
! pip install gensim
! pip install nltk
! pip install wordcloud
! pip install spacy
! pip install spacy_download
! pip install pyLDAvis
! pip install PyStemmer

! python3 -m spacy download en

/home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/venv/bin/pip: 2: exec: /home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/.venv/bin/python3: not found
/home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/venv/bin/pip: 2: exec: /home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/.venv/bin/python3: not found
/home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/venv/bin/pip: 2: exec: /home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/.venv/bin/python3: not found
/home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/venv/bin/pip: 2: exec: /home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/.venv/bin/python3: not found
/home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/venv/bin/pip: 2: exec: /home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/.venv/bin/python3: not found
/home/uji657/Downloads/Sentiment Analysis/SentiCR-master/SentiCR/venv/bin/pip: 2: exec: /h

In [1]:
## Importing PD and Others
import re
import numpy as np
import pandas as pd
from pprint import pprint

## Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

## NLTK
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk.stem
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['#', '`', '"', '@'])

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

import matplotlib.pyplot as plt
%matplotlib inline


import spacy
spacy.cli.download('en_core_web_sm')



## Visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/uji657/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/uji657/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/uji657/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 91.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Import data and Preprocess

In [2]:
df = pd.read_csv('../dataset/kepler_posts.csv')
# my_value = -1
# new_df = df.loc[df["RatingsGPTFineTuned"] == my_value]
df["merged"] = df[["Body"]]
#new_df.head()
# new_df.to_csv('Dataset/ConcatenatedDatasetSO.csv')

data = df.merged.values.tolist()
print(len(df))
#data

142


In [3]:
# Remove Emails
data = [re.sub('<[^<>]*>', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:2])

['It is not a strictly programming question, but knowing hardware is the '
 'critical part of programming. So I start this thread, hope people here can '
 'share their experience in programming on Kepler (GK10X or GK110). First I '
 'start mine: I am doing some programming on GK110 at the moment, for some '
 'applications, GK110 is significantly faster than a Fermi, close to its '
 'theoretical peak (e.g. 2.5-3X faster). But for others, it isnt, (e.g. only ~ '
 '50%-60% faster). Correct me if I am wrong, but it seems to me, the main '
 'performance bottleneck of Kepler is resource pressure is very high here: On '
 'a per-SM level, Fermi actually have far more resources comparing to GK110, '
 'on each SM, Fermi only has one SIMT unit, whilst Kepler has 6. Yet on each '
 'SM, Fermi has 32K registers file, a maximum of 1536 active threads, whilst '
 'on each SM of Kepler, there are only 33% more active threads, 100% more '
 'registers, with 800% insturction-issue units, and same amount of

# Tokenize words and Clean-up text

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['it', 'is', 'not', 'strictly', 'programming', 'question', 'but', 'knowing', 'hardware', 'is', 'the', 'critical', 'part', 'of', 'programming', 'so', 'start', 'this', 'thread', 'hope', 'people', 'here', 'can', 'share', 'their', 'experience', 'in', 'programming', 'on', 'kepler', 'gk', 'or', 'gk', 'first', 'start', 'mine', 'am', 'doing', 'some', 'programming', 'on', 'gk', 'at', 'the', 'moment', 'for', 'some', 'applications', 'gk', 'is', 'significantly', 'faster', 'than', 'fermi', 'close', 'to', 'its', 'theoretical', 'peak', 'faster', 'but', 'for', 'others', 'it', 'isnt', 'only', 'faster', 'correct', 'me', 'if', 'am', 'wrong', 'but', 'it', 'seems', 'to', 'me', 'the', 'main', 'performance', 'bottleneck', 'of', 'kepler', 'is', 'resource', 'pressure', 'is', 'very', 'high', 'here', 'on', 'per', 'sm', 'level', 'fermi', 'actually', 'have', 'far', 'more', 'resources', 'comparing', 'to', 'gk', 'on', 'each', 'sm', 'fermi', 'only', 'has', 'one', 'simt', 'unit', 'whilst', 'kepler', 'has', 'yet', 'on

In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

# Remove Stopwords, Make Bigrams and Lemmatize

In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['strictly', 'program', 'question', 'know', 'hardware', 'critical', 'part', 'programming', 'start', 'thread', 'hope', 'people', 'share', 'experience', 'programming', 'kepler', 'gk', 'first', 'start', 'mine', 'programming', 'moment', 'application', 'significantly', 'fast', 'fermi', 'close', 'theoretical', 'peak', 'fast', 'other', 'fast', 'correct', 'wrong', 'seem', 'main', 'performance', 'bottleneck', 'resource', 'pressure', 'high', 'sm', 'level', 'fermi', 'actually', 'far', 'resource', 'compare', 'fermi', 'simt', 'unit', 'yet', 'sm', 'fermi', 'register', 'file', 'maximum', 'active', 'thread', 'active', 'thread', 'register', 'insturction', 'issue', 'unit', 'amount', 'cache', 'latency', 'regard', 'memory', 'computation', 'absolute', 'term', 'half', 'term', 'cycle', 'resource', 'pressure', 'much', 'high', 'compare', 'gf', 'instruction', 'issue', 'unit', 'seem', 'want', 'use', 'aggressive', 'tlp', 'hide', 'latency', 'kepler', 'certainly', 'flexible', 'active', 'thread', 'increase', 'inste

# Create the Dictionary and Corpus needed for Topic Modeling

In [8]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 3), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 3), (26, 4), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 2), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 3), (45, 5), (46, 1), (47, 3), (48, 1), (49, 2), (50, 2), (51, 1), (52, 1), (53, 2), (54, 2), (55, 1), (56, 1), (57, 2), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 2), (65, 1), (66, 3), (67, 1), (68, 1), (69, 2), (70, 3), (71, 1), (72, 1), (73, 2), (74, 2), (75, 3), (76, 2), (77, 2), (78, 2), (79, 1), (80, 1), (81, 2), (82, 2), (83, 1), (84, 4), (85, 1), (86, 5), (87, 1), (88, 1), (89, 1), (90, 2), (91, 1), (92, 1), (93, 1), (94, 1)]]


In [9]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('absolute', 1),
  ('active', 3),
  ('actually', 1),
  ('advantage', 1),
  ('aggressive', 1),
  ('amount', 1),
  ('application', 1),
  ('available', 1),
  ('begin', 1),
  ('bottleneck', 1),
  ('cache', 1),
  ('certainly', 2),
  ('choose', 1),
  ('close', 1),
  ('code', 1),
  ('compare', 2),
  ('computation', 1),
  ('computationally', 1),
  ('contain', 1),
  ('correct', 1),
  ('critical', 1),
  ('cycle', 1),
  ('efficiency', 1),
  ('experience', 1),
  ('far', 1),
  ('fast', 3),
  ('fermi', 4),
  ('file', 1),
  ('first', 2),
  ('flexible', 1),
  ('gf', 1),
  ('gk', 1),
  ('half', 1),
  ('hard', 1),
  ('hardware', 1),
  ('hide', 2),
  ('high', 2),
  ('hope', 1),
  ('ilp', 1),
  ('increase', 1),
  ('instead', 1),
  ('instruction', 2),
  ('insturction', 1),
  ('intensive', 1),
  ('issue', 3),
  ('kepler', 5),
  ('know', 1),
  ('latency', 3),
  ('less', 1),
  ('level', 2),
  ('lot', 2),
  ('main', 1),
  ('massive', 1),
  ('maximum', 2),
  ('memory', 2),
  ('mine', 1),
  ('moment', 1),
  ('

In [11]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [12]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('../dataset/lda_tuning_results.csv', index=False)
    pbar.close()

100%|██████████| 540/540 [05:53<00:00,  1.53it/s]


In [10]:
results = pd.read_csv('../dataset/lda_tuning_results.csv')
results = results.loc[272:]
results.sort_values(by=['Coherence'], ascending=False)

,Validation_Set,Topics,Alpha,Beta,Coherence
297,100% Corpus,2,asymmetric,0.61,0.544267
299,100% Corpus,2,asymmetric,symmetric,0.544267
298,100% Corpus,2,asymmetric,0.9099999999999999,0.542629
411,100% Corpus,6,symmetric,0.31,0.537383
391,100% Corpus,6,0.01,0.31,0.537121
...,...,...,...,...,...
528,100% Corpus,10,0.9099999999999999,0.9099999999999999,0.434032
423,100% Corpus,7,0.01,0.9099999999999999,0.431641
453,100% Corpus,8,0.01,0.9099999999999999,0.425680
473,100% Corpus,8,symmetric,0.9099999999999999,0.422407


In [14]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=100,
                                           alpha=0.61,
                                           eta=0.61)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


Perplexity:  -6.75086639612273

Coherence Score:  0.3793164665020995


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.145984  0.089596       1        1  55.087426
6      0.064695 -0.109529       2        1  11.785881
2     -0.125553  0.047679       3        1   9.321528
4     -0.018923 -0.006536       4        1   8.300281
0      0.016301 -0.016944       5        1   7.226738
1     -0.042485  0.010316       6        1   4.934322
5     -0.040019 -0.014582       7        1   3.343824, topic_info=         Term        Freq       Total Category  logprob  loglift
289       org  151.000000  151.000000  Default  30.0000  30.0000
97    eclipse  389.000000  389.000000  Default  29.0000  29.0000
218      java  110.000000  110.000000  Default  28.0000  28.0000
551      info   53.000000   53.000000  Default  27.0000  27.0000
485      name   68.000000   68.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
27       file    1.491006  105.206790   Topic7  -5.6527  -0.8584
224       add    1.149042   26.583765   Topic7  -5.9132   0.2567
488  property    1.078624   18.684931   Topic7  -5.9765   0.5460
221      show    1.077336   22.219212   Topic7  -5.9777   0.3716
89       want    1.042284   17.456058   Topic7  -6.0108   0.5798

[390 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
1430      1  0.206960  abego_treelayout
1430      5  0.620880  abego_treelayout
2058      1  0.420359       accordingly
2058      7  0.420359       accordingly
654       1  0.888681          activate
...     ...       ...               ...
1238      6  0.140861             write
1456      1  0.272233       yang_parser
1456      5  0.544467       yang_parser
1707      1  0.136783             zkoss
1707      5  0.820701             zkoss

[842 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 7, 3, 5, 1, 2, 6])

In [16]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print(doc_lda)

[(0,
  '0.029*"org" + 0.020*"opendaylight_yangtool" + 0.010*"com_google" + '
  '0.007*"eclipse" + 0.006*"feature" + 0.006*"zkoss" + 0.006*"dependency" + '
  '0.006*"install" + 0.006*"studio" + 0.005*"read_repository"'),
 (1,
  '0.012*"kepler_gl" + 0.008*"store" + 0.008*"try" + 0.008*"feature" + '
  '0.008*"true" + 0.007*"plexus" + 0.006*"type" + 0.006*"use" + '
  '0.006*"latitude_longitude" + 0.005*"component"'),
 (2,
  '0.022*"datum" + 0.019*"name" + 0.017*"none" + 0.017*"self" + '
  '0.011*"format_type" + 0.010*"false" + 0.010*"python_site" + 0.010*"color" + '
  '0.009*"type" + 0.009*"map"'),
 (3,
  '0.054*"eclipse" + 0.027*"internal" + 0.021*"error" + 0.016*"run" + '
  '0.015*"org" + 0.013*"file" + 0.013*"project" + 0.012*"java" + 0.011*"core" '
  '+ 0.009*"try"'),
 (4,
  '0.012*"use" + 0.008*"code" + 0.006*"problem" + 0.006*"try" + '
  '0.005*"connection" + 0.004*"heap" + 0.004*"result" + 0.004*"still" + '
  '0.004*"variable" + 0.004*"bin_node"'),
 (5,
  '0.008*"use" + 0.007*"map" 

In [17]:
num_topics = 10
topic_distributions = [lda_model[doc] for doc in corpus]

# Extract the dominant topic for each document
df['topic'] = [max(topics, key=lambda x: x[1])[0] for topics in topic_distributions]

# Display the DataFrame with assigned topics
print(df.head(3))
df.to_csv('../dataset/AssignedTopicKepler.csv')
for i in range(num_topics): 
    topic_df = df[df['topic'] == i]
    print('topic: ', i, 'total data: ', len(topic_df))

         Id                                              Title  \
0  15466642  What are the limiting factors of Kepler, compa...   
1  17083811    Eclipse - Glassfish 4 Debugging startup timeout   
2  17336260  Eclipse Kepler - Glassfish Plugin - unable to ...   

                                                Body  RatingsSentiCR  \
0  It is not a strictly programming question, but...               1   
1  Glassfish 4.0 final gives a timeout everytime ...              -1   
2  I'm using Eclipse Kepler WTP + the Glassfish K...              -1   

   RatingsGPT35  RatingsGPTFineTuned  \
0            -1                 -1.0   
1            -1                 -1.0   
2            -1                 -1.0   

                                              merged  topic  
0  It is not a strictly programming question, but...      4  
1  Glassfish 4.0 final gives a timeout everytime ...      3  
2  I'm using Eclipse Kepler WTP + the Glassfish K...      3  
topic:  0 total data:  8
topic:  1 to